In [53]:
import bs4
import requests
import json
from random import shuffle
from thefuzz import process, fuzz
from modules.CountMinSketch.count_min_sketch import CountMinSketch
from modules.BloomFilter.bloom_filter2 import BloomFilter2
from modules.Dataset.dataset import Dataset

In [48]:
ds = Dataset()

words_list = ds.build_words_list(num_of_articles = 12)
print(len(words_list))

230449


In [49]:
len(words_list)

230449

In [50]:
NUM_COUNTERS = 200000
cms = CountMinSketch(NUM_COUNTERS)

for word in words_list:
    cms.increment(word)

In [54]:
n = len(words_list) #no of items to add 
p = 0.01 #false positive probability 
bloomf = BloomFilter2(n,p)

for word in words_list:
    bloomf.add(word)

In [60]:
#some_word_with_error = "appel"
some_sentance_with_error = "I wanna appel pleases or a bannana with a paer and a lemone, or a wartermelon if possibe. I hope i don't have an errof"
new_sentance = []
for word in some_sentance_with_error.split():
    if not word in bloomf:
        success_percentage = max((len(word) - 1) / len(word) * 100, 75)
        print(f"Scanning {word}, success_percentage is {success_percentage}")
        word_results = [x for x in process.extract(word, words_list, scorer=fuzz.ratio)]# if x[1] >= success_percentage]
        if (len(word_results) > 0 and word_results[0][1] < 100):
            list_fixes = [x[0] for x in word_results]
            for fix in list_fixes:
                print(f"Frequency of {fix} => {cms.approximateCount(fix)} and match rate is {word_results[list_fixes.index(fix)][1]}")
            #new_sentance.append(word_results[0][0] if word_results[0][1] >= success_percentage else word)
        else:
            continue
            #new_sentance.append(word)
    else:
        print(f"Word {word} is correct. Skipping")

#print(' '.join(new_sentance))

Word I is correct. Skipping
Scanning wanna, success_percentage is 80.0
Scanning appel, success_percentage is 80.0
Frequency of Appelo, => 1 and match rate is 91
Frequency of appeal => 2 and match rate is 91
Frequency of Appeal" => 1 and match rate is 91
Frequency of appeal => 2 and match rate is 91
Frequency of appeal. => 1 and match rate is 91
Scanning pleases, success_percentage is 85.71428571428571
Frequency of Please => 7 and match rate is 92
Frequency of Please => 7 and match rate is 92
Frequency of please. => 2 and match rate is 92
Frequency of Please => 7 and match rate is 92
Frequency of Please => 7 and match rate is 92
Word or is correct. Skipping
Word a is correct. Skipping
Scanning bannana, success_percentage is 85.71428571428571
Frequency of Anna-Lena => 3 and match rate is 75
Frequency of Anna-Lena => 3 and match rate is 75
Frequency of Anna-Lena => 3 and match rate is 75
Frequency of Anna => 4 and match rate is 73
Frequency of Anna => 4 and match rate is 73
Word with is c